In [304]:
using JuMP
using GLPK

In [305]:
m = Model();
set_optimizer(m, GLPK.Optimizer);

In [306]:
set_time_limit_sec(m, 1800.0);

In [307]:
function reader(filepath :: String)
  k_count = 0
  arestas = []
  v_count = 0
  
  open(filepath, "r") do f
    line = readline(f)
    line = readline(f)
    v_count = parse(Int, line) # Number of vertices
    line = readline(f)
    line = readline(f)

    while ! eof(f)
      line = readline(f)
      words = split(line)
      if isempty(words)
          continue
      end
      vertex1 = parse(Int,words[1])
      vertex2 = parse(Int,words[2])
      edge_type = parse(Int,words[3])

      if edge_type > k_count
        k_count = edge_type
      end

      push!(arestas, (vertex1,vertex2,edge_type))
    end
  end
  return (arestas, k_count, v_count)
end

reader (generic function with 1 method)

In [308]:
filepath = "/home/leo/Documents/2021_1/Otimizacao/Trabalho/final-otimizacao/rm/jcole.txt"

my_tuple = reader(filepath)
arestas = my_tuple[1]
K = my_tuple[2]
V = my_tuple[3];

In [309]:
function create_arestas_k(arestas, k)
    arestas_k = [[] for i=1:k]
    for a in arestas
        push!(arestas_k[a[3]], (a))
    end
    return arestas_k
end

create_arestas_k (generic function with 1 method)

In [310]:
arestas_k = create_arestas_k(arestas,K);

In [311]:
@variable(m, x[i in arestas], Bin);

In [312]:
@constraint(m, [k in 1:K], sum(x[i] for i in arestas_k[k]) <=1);

In [313]:
function create_arestas_v(arestas, v)
    arestas_v = [[] for i=1:v]
    for a in arestas
        push!(arestas_v[a[1]], (a))
        push!(arestas_v[a[2]], (a))
    end
    return arestas_v
end

create_arestas_v (generic function with 1 method)

In [314]:
arestas_v = create_arestas_v(arestas,V);

In [315]:
@constraint(m, [v in 1:V], sum(x[i] for i in arestas_v[v]) <=1);

In [316]:
@objective(m, Max, sum(x[i] for i in arestas));

In [317]:
optimize!(m)

In [318]:
objective_value(m)

2.0

In [319]:
@show value.(x)

value.(x) = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, Any[(1, 2, 4), (1, 3, 1), (1, 4, 1), (2, 3, 3), (2, 4, 1), (2, 5, 1)]
And data, a 6-element Vector{Float64}:
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, Any[(1, 2, 4), (1, 3, 1), (1, 4, 1), (2, 3, 3), (2, 4, 1), (2, 5, 1)]
And data, a 6-element Vector{Float64}:
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0

In [320]:
using DelimitedFiles

open("answer.txt", "w") do f
    writedlm(f, value.(x))

end